In [111]:
# Import Libraries

import pandas as pd
import nltk
import pymorphy2
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ilyin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ilyin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ilyin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [112]:
df = pd.read_csv('test_data.csv')

greeting = ['здравствуйте','добрый','доброе','доброго','привет','здрасте','приветствую']
goodbye = ['досвидания','до свидания','хорошего','хороших','доброго','свидания','встречи','связи']
present = ['меня','зовут','имя','это','менеджер']


In [113]:
item_a = []
item_b = []
item_c = []
item_d = []
item_e = []
item_f = []
dialogs = []

for i in range(df.loc[len(df)-1, "dlg_id"]+1):
    names = []
    dialogs.append(str(i+1))
    prob_thresh = 0.4
    morph = pymorphy2.MorphAnalyzer()

    dialog = df[df["dlg_id"] == i]
    manager = dialog[dialog["role"] == 'manager']

    total_list = []

    replicas = []
    full_text = ''
    text = manager.text.to_list()
    for k in text:
        replic = k.lower()
        total_list += replic.split(' ')
        replicas.append(replic.split(' '))
        full_text += replic
        full_text += ' '

    # Finding greeting

    for r in range(5):
        crossover = list(set(replicas[r]) & set(greeting))
        if len(crossover) > 0:
            item_a.append(text[r])
            break
        if r == 4 and len(crossover) == 0:
            item_a.append('Отсутствует')

    # Present

    names = []
    for word in nltk.word_tokenize(full_text):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                names.append(word)
    for j in range(len(replicas)):
        list1 = list(set(replicas[j]) & set(names))
        list2 = list(set(replicas[j]) & set(present))
        if len(list1) > 0 and len(list2) > 0:
            item_b.append(text[j])

            # Manager's name
            indexes1 = []
            indexes2 = []
            for l1 in list1:
                indexes1.append(replicas[j].index(l1))
            for l2 in list2:
                indexes2.append(replicas[j].index(l2))

            for n in indexes1:
                post_check1 = 0
                post_check2 = 0
                for t in indexes2:
                    if n > t:
                        post_check1 +=1
                    if abs(n-t) < 2:
                        post_check2 +=1
                if post_check1 > 0 and post_check2 > 0:
                    item_c.append(str(replicas[j][n]))

    if len(item_b) != len(item_a):
        item_b.append('Отсутствует')
    if len(item_c) != len(item_a):
        item_c.append('Отсутствует')

    # Company's name
    company_name = ''
    global check1
    check1 = 0
    try:
        global namelen
        namelen = 1
        number = total_list.index('компания')
        company_name += total_list[number+1]
        company_name += ' '
        while check1 == 0:
            if str(morph.parse(total_list[number+1+namelen])[0].tag.POS) == 'NOUN' and morph.parse(total_list[number+1+namelen])[0].score >= 0.35:
                company_name += total_list[number+1+namelen]
                company_name += ' '
                namelen += 1
            elif morph.parse(total_list[number+1+namelen])[0].score < 0.3:
                company_name += total_list[number+1+namelen]
                company_name += ' '
                namelen += 1
            elif str(morph.parse(total_list[number+1+namelen])[0].tag.POS) == 'CONJ':
                company_name += total_list[number+1+namelen]
                company_name += ' '
                namelen += 1
            elif str(morph.parse(total_list[number+1+namelen])[0].tag.POS) == 'None':
                company_name += total_list[number+1+namelen]
                company_name += ' '
                namelen += 1
            else:
                check1 = 1
        else:
            namelen = 1
            item_d.append(company_name)
    except:
        item_d.append('Отсутствует')

    # Finding bye
    len_dialog = len(replicas)
    for s in range(len_dialog-3,len_dialog):
        crossover1 = list(set(replicas[s]) & set(goodbye))
        if len(crossover1) > 0:
            item_e.append(text[s])
            break
        if s == len_dialog-1 and len(crossover1) == 0:
            item_e.append('Отсутствует')

for b in range(len(item_a)):
    if item_a[b] != 'Отсутствует' and  item_e[b] != 'Отсутствует':
        item_f.append(True)
    else:
        item_f.append(False)


In [114]:
df_itog = pd.DataFrame({'Диалог':dialogs,
                        'Приветствием':item_a,
                        'Реплики, где менеджер представил себя':item_b,
                        'Имя менеджера':item_c,
                        'Название компании':item_d,
                        'Прощание':item_e,
                        'Требование к менеджеру':item_f})